# Tâche 2 : Question-réponse avec modèle pré‑entraîné GPT‑2, sans adaptation

**Objectifs**

Observer les limites d’un modèle de langage **pré‑entraîné** (**GPT-2 Medium (355M)**) lorsqu’on lui pose des questions sur un sujet **absent** de ses données d’origine (ici, l’univers de *Sherlock Holmes*).
Dans ce *notebook*, vous faites **uniquement de l’inférence** avec un modèle qui a déjà été pré-entraîné. Cette tâche consiste à construire un *prompt* minimal, générer des réponses avec le modèle sans modification, puis évaluer la pertinence des résultats. Aucun entraînement de modèle n'est effectué pour cette tâche.

**Objectifs d’apprentissage**
1. Charger un modèle pré‑entraîné et son tokenizer de Hugging Face.
2. Générer du texte et **isoler la réponse** du modèle.
3. Comprendre et expliquer les **limites du pré‑entraînement** hors‑domaine.

Les **questions** pour évaluer le modèle vous sont fournies. Vous devez comprendre le format des questions chargées en mémoire.

> Il est recommandé de faire ce travail pratique en utilisant une carte graphique GPU compatible avec HuggingFace/Pytorch.
> Si votre machine n’en possède pas, vous pouvez utiliser **Google Colab** pour exécuter le *notebook* dans le cloud.

Si nécessaire, installer les *packages* suivant. Si vous exécutez sur Code Colab, ces *packages* devraient déjà être installés.

In [ ]:
#!pip install datasets
#!pip install accelerate
#!pip install transformers[torch]
#!pip3 install torch torchvision

In [1]:
batch_size = 5 # il est possible d'ajuster la taille de batch. Les valeurs actuelles utilisent environ 10 Gb
max_length = 256 # on réduit le contexte pour sauver du temps, nos exemples ne nécesside pas une plus grande fenêtre de mots
model_name = "gpt2-medium"

In [2]:
from transformers import pipeline
import os
import json

## 1. Chargement du modèle Hugging Face et du tokenizer (à compléter)

Complétez le corps de la fonction `load_model(model_path)` afin qu’elle :

- charge le **tokenizer** et le **modèle** Hugging Face à partir du chemin `model_path`.
- **retourne** le tokenizer comme **première valeur de retour** et le modèle comme **seconde valeur de retour**.

On ajoute également des fonctions pour monter les questions en mémoire et pour sauvegarder les réponses dans un fichier.

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM

def load_model(model_path):
    tokenizer = AutoTokenizer.from_pretrained(model_path)
    model = AutoModelForCausalLM.from_pretrained(model_path)

    # GPT-2 n'en a pas par défaut
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
        model.config.pad_token_id = tokenizer.eos_token_id

    return tokenizer, model

In [4]:
def load_entries(path):
    with open(path, "r", encoding="utf-8") as file:
        data = json.load(file)
    if not isinstance(data, list):
        raise ValueError(f"Question file must contain a list of objects. Got: {type(data)}")
    return data

def save_answers(questions_answers, output_dir, out_file_name, display=True):
    os.makedirs(output_dir, exist_ok=True)
    with open(os.path.join(output_dir, out_file_name), "w", encoding="utf-8") as out:
        for index, question, answer, expected_answer in questions_answers:
            out.write(f"Q: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}\n")
            if display:
                print(f"Q{index}: {question}\nA: {answer}\nExpected:{expected_answer}\n{'-' * 60}")


## 2. Fonctions de test question-réponse  (à compléter)

La fonction **test_on_questions** est utilisée pour parcourir **toutes les entrées** du fichier de questions afin de produire des réponses générées par le modèle.

La génération d'une réponse à une question implique les étapes suivantes (fonction **process_entry** à compléter) :
* Construire un prompt à l’aide de la fonction **build_prompt** (rendu disponible).
* Utiliser le modèle (via un pipeline de génération passé en argument) pour générer une réponse à une question.
* Retourner la réponse générée par le modèle.  

Points importants à souligner:
* La fonction ***process_entry*** doit retourner uniquement la réponse générée par le modèle (sans la question - le prompt).
* Il est de votre responsabililté de choisir **les paramètres** du générateur (max_new_tokens, do_sample, temperature, top_k ou top_p). Décrivez ceux que vous avez retenus.

> Afin de simplifier le travail, nous avons choisi de ne pas utiliser de *batchs* dans la fonction qui teste les questions.
> Vous n'avez pas à prendre en compte le *warning* qui suggère d'utiliser des *datasets*.

In [5]:
def build_prompt(entry):
    return entry.get("question", "")

### Description des paramètres de génération:
- **max_new_tokens=100** : Limite la longueur de la réponse à 100 tokens pour éviter des réponses trop longues
- **do_sample=True** : Active l'échantillonnage probabiliste pour des réponses plus variées et naturelles
- **temperature=0.7** : Contrôle la créativité (0.7 offre un bon équilibre entre cohérence et diversité)
- **top_p=0.9** : Considère les tokens dont la probabilité cumulative atteint 90%
- **top_k=50**: Limite la sélection aux 50 tokens les plus probables à chaque étape
- **pad_token_id** : Nécessaire pour éviter les warnings avec GPT-2

In [15]:
def process_entry(entry, prompt_builder, generator):
    # Construit le prompt à partir de l'entrée
    prompt = prompt_builder(entry)

    # Génére la réponse avec le modèle
    result = generator(
        prompt,
        max_new_tokens=100,       # Longueur maximale de la réponse
        do_sample=True,           # Active l'échantillonnage probabiliste
        temperature=0.7,          # Contrôle la créativité
        top_p=0.9,                # Échantillonnage nucleus
        top_k=100,                # Échantillonnage top-k
        pad_token_id=generator.tokenizer.eos_token_id  # Évite les warnings
    )

    # Extrait le texte généré complet
    generated_text = result[0]['generated_text']

    # Retourne uniquement la réponse (enleve le prompt)
    answer = generated_text[len(prompt):].strip()
    return answer

In [9]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [10]:
def test_on_questions(prompt_builder, model_path, question_file, out_file_name, output_dir="results"):
    entries = load_entries(question_file)
    tokenizer, model = load_model(model_path)
    generator = pipeline("text-generation", model=model, tokenizer=tokenizer)
    results = []
    for i, entry in enumerate(entries):
        answer = process_entry(entry, prompt_builder, generator)
        question = entry.get("question", "")
        expected_answer = entry.get("answer", "")
        results.append((i, question, answer, expected_answer))
    save_answers(results, output_dir, out_file_name, display=True)
    return results

## 3. Génération avec GPT-2 de réponses aux questions sur Sherlock Holmes

Exécutez la cellule suivante pour générer les réponses aux questions.
Le temps d’exécution devrait se situer entre **5 et 10 minutes** si vous utilisez **Google Colab** avec un GPU.

Note : N'oubliez pas d'ajouter le fichier de réponses générées par le modèle (voir *out_file_name*) dans votre remise du travail. Ainsi le fichier ZIP que vous déposerez sur le site du cours devra contenir tous les *notebooks* et tous les fichiers de réponses.

In [16]:
questions = "data/questions_sherlock.json"
out_file_name = "gpt2_answers.txt"

results = test_on_questions(prompt_builder=build_prompt, model_path=model_name, question_file=questions, out_file_name=out_file_name)

Device set to use cuda:0


Q0: Where do Sherlock Holmes and Dr. Watson live?
A: It's a good question. Sherlock Holmes lived in a hotel in London, which is not exactly the place to start a novel. He lived in the hotel, of course. But then, the hotel was not the place to start a novel, so Holmes lived in a boarding house, in which he was kept in a perpetual state of stasis, and which he had to rely on his faithful companion, Watson. (Watson is a bit of a mystery, since he seems to be
Expected:221B Baker Street, London.
------------------------------------------------------------
Q1: Who is Sherlock Holmes' loyal friend and chronicler?
A: Who is his rival? Who is his nemesis? What is his ultimate goal?

Sherlock Holmes is a serial killer, a mad scientist, a spy, a psychiatrist, a detective, a spy, a killer, a philosopher, a detective, a spy, a killer, a spy, a spy, a spy, a spy, a spy, a spy, a spy, a spy, a spy, a spy, a spy, a spy, a spy, a spy,
Expected:Dr. John H. Watson.
---------------------------------------

### Choix de max_new_tokens=100

Au début, on a testé avec 500 tokens pour laisser le modèle générer des réponses plus longues. Malheureusement, les résultats n'étaient pas satisfaisants. Le modèle se mettait à répéter les mêmes phrases en boucle, comme s'il ne savait plus quoi dire. Ce comportement répétitif rendait les réponses difficiles à exploiter.

En plus, les réponses s'éloignaient beaucoup trop du sujet. Le modèle inventait des histoires complètes qui n'avaient rien à voir avec la question posée. Ça produisait beaucoup de texte, mais très peu d'informations pertinentes.

**On a donc choisi 100 tokens à la place.** C'est un meilleur compromis parce que:
- Les réponses restent courtes et faciles à analyser
- Le modèle s'arrête avant de partir hors-sujet
- C'est suffisant pour qu'il puisse formuler une réponse complète
- Ça correspond mieux aux vraies réponses attendues (qui sont généralement très courtes)

Avec 100 tokens, on observe mieux les vraies capacités du modèle sur ce domaine, sans avoir trop de texte inutile à filtrer.

## 4. Analyse des résultats

### 4.1 Évaluation quantitative (à compléter)

In [17]:
import string
import re
from collections import Counter

def remove_articles(text):
    return re.sub(r'\b(a|an|the)\b', ' ', text)

def white_space_fix(text):
    return ' '.join(text.split())

def remove_punc(text):
    exclude = set(string.punctuation)
    return ''.join(ch for ch in text if ch not in exclude)

def lower(text):
    return text.lower()

def normalize_answer(s):
    """Mettre en minuscule et retirer la ponctuation, des déterminants and les espaces."""
    return white_space_fix(remove_articles(remove_punc(lower(s))))

In [18]:
def evaluate_f1(ground_truth, prediction):
    """Normalise les 2 textes, trouve ce qu'il y a en commun et estime précision, rappel et F1."""
    prediction_tokens = normalize_answer(prediction).split()
    ground_truth_tokens = normalize_answer(ground_truth).split()
    common = Counter(prediction_tokens) & Counter(ground_truth_tokens)
    num_same = sum(common.values())
    if num_same == 0:
        return 0.0, 0.0, 0.0
    precision = 1.0 * num_same / len(prediction_tokens)
    recall = 1.0 * num_same / len(ground_truth_tokens)
    f1 = (2 * precision * recall) / (precision + recall)
    return precision, recall, f1

In [19]:
def evaluation_generation(results):
    total_precision = 0
    total_recall = 0
    total_f1 = 0
    num_questions = len(results)

    # Calcule les métriques pour chaque question
    for i, question, answer, expected_answer in results:
        precision, recall, f1 = evaluate_f1(expected_answer, answer)
        total_precision += precision
        total_recall += recall
        total_f1 += f1

    # Calcule les moyennes
    eval = {
        'precision_moyenne': total_precision / num_questions,
        'rappel_moyen': total_recall / num_questions,
        'f1_moyen': total_f1 / num_questions,
        'nombre_questions': num_questions
    }

    return eval

In [20]:
eval = evaluation_generation(results)
print(eval)

{'precision_moyenne': 0.004469136364648286, 'rappel_moyen': 0.10071428571428571, 'f1_moyen': 0.008330692946605546, 'nombre_questions': 50}


**Question:** Que pensez-vous de cette évaluation ?

- Les scores F1, précision et rappel sont extrêmement faibles (F1 = 0.8%), ce qui confirme quantitativement ce que l'analyse qualitative des réponses révèle : GPT-2 échoue presque complètement à répondre correctement aux questions sur Sherlock Holmes.

- Le rappel moyen de 10% indique que le modèle capture quelques mots communs des réponses attendues, mais la précision quasi-nulle (0.4%) montre qu'il génère énormément de texte superflu et incorrect.

- Le modèle génère du texte grammaticalement correct et apparemment cohérent, mais qui est faux, ce qui illustre le problème d'hallucination des LLMs sans connaissances spécifiques (pour atteindre le nombre de tokens demandés, il rajoute lui-même des questions et invente des réponses).


### 4. Analyse qualitative (à faire)

Rédigez **5 à 8 phrases** expliquant ce que vous observez et pourquoi, selon vous, le modèle fournit ce type de réponses.

> Cette étape prépare le terrain pour les tâches 3 et 4.
> Il est normal que les réponses ne soient pas très bonnes à ce stade. On vous demande d’expliquer **pourquoi**.

Les résultats montrent que GPT-2 génère des réponses complètement incorrectes sur l'univers de Sherlock Holmes. Par exemple, le modèle invente que Holmes et Watson habitent dans un hôtel à Londres au lieu de 221B Baker Street à Londres. Ce problème s'explique par le fait que GPT-2 n'a pas été entraîné spécifiquement sur les textes de Sherlock Holmes. Le modèle essaie quand même de générer des réponses en se basant uniquement sur les structures de phrases qu'il a apprises, ce qui produit du texte qui semble correct grammaticalement mais qui est faux. On observe aussi que le modèle invente beaucoup de détails (noms, dates, lieux) qui semblent vrais mais n'existent pas. Le score F1 très faible de 0.8% confirme cette mauvaise performance. Ces observations montrent qu'un modèle pré-entraîné de façon générale ne peut pas répondre correctement à des questions précises sur un domaine qu'il ne connaît pas.

